1. MOUNT DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Block 2: Install & Import Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Block 3: Set Paths & Read CSVs

In [3]:

# Training + validation data
train_dir = "/content/drive/MyDrive/Colab Notebooks/p2-dataset/training images/train"
train_labels_file = "/content/drive/MyDrive/Colab Notebooks/p2-dataset/train.csv"

labels_df = pd.read_csv(train_labels_file)
labels_df.columns = labels_df.columns.str.strip()
labels_df['Label'] = labels_df['Label'].astype(str)

print(labels_df.head())
print("Total training rows:", len(labels_df))
print("Files in folder:", len(os.listdir(train_dir)))

# Test data
test_dir = "/content/drive/MyDrive/Colab Notebooks/p2-dataset/testing image/test"
test_labels_file = "/content/drive/MyDrive/Colab Notebooks/p2-dataset/test.csv"

test_labels_df = pd.read_csv(test_labels_file)
test_labels_df.columns = test_labels_df.columns.str.strip()

# Removed: test_labels_df['Label'] = test_labels_df['Label'].astype(str)
print(test_labels_df.head())
print("Total test rows:", len(test_labels_df))
print("Files in test folder:", len(os.listdir(test_dir)))

   Image Label
0  1.jpg     0
1  2.jpg     0
2  3.jpg     1
3  4.jpg     1
4  5.jpg     0
Total training rows: 945
Files in folder: 945
     Image
0  946.jpg
1  947.jpg
2  948.jpg
3  949.jpg
4  950.jpg
Total test rows: 402
Files in test folder: 402


Block 4: Preprocessing & Augmentation

In [12]:
IMG_SIZE = (299, 299)

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.1
)

# Training generator
train_gen = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=train_dir,
    x_col='Image',
    y_col='Label',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

# Validation generator
val_gen = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=train_dir,
    x_col='Image',
    y_col='Label',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Test generator (no augmentation or labels)
test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_labels_df,
    directory=test_dir,
    x_col='Image',
    y_col=None, # Removed 'Label' as it doesn't exist in test_labels_df
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode=None, # Set to None for prediction without labels
    shuffle=False
)

Found 756 validated image filenames belonging to 2 classes.
Found 189 validated image filenames belonging to 2 classes.
Found 402 validated image filenames.


Block 5: Load Pretrained Xception

In [13]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299,299,3))

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

Block 6: Compile Model

In [14]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer_2[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 74, 74,    │        512 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add_24[0][0]    

 Total params: 21,911,081 (83.58 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Block 7: Train Model

In [15]:
# Re-compile the model to ensure it's in a compiled state before training
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=23,
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 202s 7s/step - accuracy: 0.5820 - loss: 0.6749 - val_accuracy: 0.7037 - val_loss: 0.6232
Epoch 2/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.7076 - loss: 0.6113 - val_accuracy: 0.7831 - val_loss: 0.5729
Epoch 3/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.7449 - loss: 0.5785 - val_accuracy: 0.7725 - val_loss: 0.5410
Epoch 4/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.7705 - loss: 0.5336 - val_accuracy: 0.7937 - val_loss: 0.5130
Epoch 5/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.8146 - loss: 0.5022 - val_accuracy: 0.8307 - val_loss: 0.4639
Epoch 6/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.7963 - loss: 0.4789 - val_accuracy: 0.8307 - val_loss: 0.4722
Epoch 7/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 0.8331 - loss: 0.4581 - val_accuracy: 0.8254 - val_loss: 0.4365
Epoch 8/23
24/24 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.8149 - loss: 0.4465 - val_accuracy: 0.8466 - val_loss

Block 8: Save model

In [16]:
# Save the trained model
model.save("/content/drive/MyDrive/ai_real_model.h5")
print("Model saved successfully!")

Model saved successfully!


Block 9: Evaluate on Test Set

In [17]:
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc*100:.2f}%")


ValueError: None values not supported.

Block 9: Get Predictions

In [ ]:
preds = model.predict(test_gen)
pred_labels = (preds > 0.5).astype(int)

results = pd.DataFrame({
    "Image": test_labels_df['Image'],
    "True_Label": test_labels_df['Label'],
    "Predicted_Label": pred_labels.flatten()
})

results.head()

Block 10: Plot Training History

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss')
plt.show()
